In [2]:
import pandas as pd
import csv
import json

In [9]:
nwards = 24

masterfile = "Mumbai_master.xlsx"

df = pd.read_excel(masterfile,sheet_name="Demographics")
df = df.head(nwards)


wardLetterToNo = {}

wardNoToLetter = {}
for index, row in df.iterrows():
    wardLetterToNo[row["BMC_ID"]] = row["wardNo"]
    wardNoToLetter[row["wardNo"]] = row["BMC_ID"]
    
    wardLetterToNo[row["BMC_ID"]+"_hd"] = row["wardNo"]+nwards
    wardNoToLetter[row["wardNo"]+nwards] = row["BMC_ID"]+"_hd"
df

,wardIndex,wardNo,BMC_ID,wardName,area.sq.km.,totalPopulation,Households,slumDweFrac (old),slumDweFrac,slumDwePop,...,FemalePop,MalePop,TotalWorkerPercentage,#TotalWorkers,#TotalSlumworkers,#Totalnonslumworkers,Zone,ZoneArea,ZonalAreaFrac,ZonPopFrac
0,0,1,A,Colaba,12.5,185014,43866,0.289,0.120434,22282,...,83802.8,101211,45.3,83811.3,10093.7,73717.6,1,30.8,0.405844,0.151823
1,1,2,B,Sandhurst Road,2.5,127290,27140,0.133,0.0998586,12711,...,56847.3,70442.7,39.5,50279.6,5020.85,45258.7,1,30.8,0.0811688,0.104454
2,2,3,C,Marine Lines,1.8,166161,36479,0,0.0997286,16571,...,68130.9,98030.1,47.7,79258.8,7904.37,71354.4,1,30.8,0.0584416,0.136352
3,3,4,D,Grant Road,6.6,346866,77556,0.099,0.100036,34699,...,163920,182946,44.1,152968,15302.3,137666,1,30.8,0.214286,0.284639
4,4,5,E,Byculla,7.4,393286,78346,0.119,0.315785,124194,...,177195,216091,40.6,159674,50422.8,109251,1,30.8,0.24026,0.322731
5,5,6,F/N,Matunga,13,529034,119624,0.581,0.450119,238128,...,243840,285194,39.3,207910,93584.3,114326,2,46.1,0.281996,0.283392
6,6,7,F/S,Parel,14,360972,79733,0.358,0.499008,180128,...,170183,190789,39.1,141140,70430,70710,2,46.1,0.303688,0.193365
7,7,8,G/N,Dadar/Plaza,10,599039,128138,0.558,0.603757,361674,...,267895,331144,41.7,249799,150818,98981.2,2,46.1,0.21692,0.320892
8,8,9,G/S,Elphinstone,9.1,377749,83457,0.331,0.32907,124306,...,168933,208816,42.1,159032,52332.8,106700,2,46.1,0.197397,0.202352
9,9,10,H/E,Khar/Santacruz,13.5,557239,120266,0.788,0.697946,388923,...,251400,305839,41.4,230697,161014,69682.8,3,73.3,0.184175,0.228621


In [6]:
def checkRowOrdering(df, n, name="(unknown-df)"):
    assert df.shape[0]==n, f"{name} does not have {n} rows"
    count = 0
    for index,row in df.iterrows():
        assert row["wardIndex"] == count, f"{name} has wardIndex mismatch in row {count}"
        count+=1

In [8]:
demographics_nonhd = df[["wardIndex", "wardNo", "BMC_ID", "wardName", "area.sq.km.","nonslumDwePop"]].sort_values(
    'wardNo'
).rename(columns={"nonslumDwePop": "totalPopulation"})

demographics_hd = df[["wardIndex", "wardNo", "BMC_ID", "wardName", "area.sq.km.","slumDwePop"]].sort_values(
    'wardNo'
).rename(columns={"slumDwePop": "totalPopulation"})

demographics_hd["wardNo"] = demographics_hd["wardNo"] + 24
demographics_hd["wardIndex"] = demographics_hd["wardIndex"] + 24
demographics_hd["BMC_ID"] = demographics_hd["BMC_ID"] + "_hd"
demographics_hd["wardName"] = demographics_hd["wardName"] + "_hd"

demographics_nonhd["hd_flag"] = 0
demographics_hd["hd_flag"] = 1


demographics = pd.concat([demographics_nonhd, demographics_hd], axis=0)
demographics['totalPopulation'] = demographics['totalPopulation'].astype(int)

checkRowOrdering(demographics, 48, "demographics")

#demographics


,wardIndex,wardNo,BMC_ID,wardName,area.sq.km.,totalPopulation,hd_flag
0,0,1,A,Colaba,12.5,162732,0
1,1,2,B,Sandhurst Road,2.5,114579,0
2,2,3,C,Marine Lines,1.8,149590,0
3,3,4,D,Grant Road,6.6,312167,0
4,4,5,E,Byculla,7.4,269092,0
5,5,6,F/N,Matunga,13,290906,0
6,6,7,F/S,Parel,14,180844,0
7,7,8,G/N,Dadar/Plaza,10,237365,0
8,8,9,G/S,Elphinstone,9.1,253443,0
9,9,10,H/E,Khar/Santacruz,13.5,168316,0


In [10]:
households_nonhd = df[["wardIndex", "wardNo", "BMC_ID", "wardName", "Households"]].sort_values('wardNo')
households_nonhd["Households"] = -1

households_hd = households_nonhd.copy()

households_hd["wardNo"] = households_hd["wardNo"] + 24
households_hd["wardIndex"] = households_hd["wardIndex"] + 24
households_hd["BMC_ID"] = households_hd["BMC_ID"] + "_hd"
households_hd["wardName"] = households_hd["wardName"] + "_hd"

households_nonhd["hd_flag"] = 0
households_hd["hd_flag"] = 1


households = pd.concat([households_nonhd,households_hd], axis=0)

checkRowOrdering(households, 48, "households")


households


,wardIndex,wardNo,BMC_ID,wardName,Households,hd_flag
0,0,1,A,Colaba,-1,0
1,1,2,B,Sandhurst Road,-1,0
2,2,3,C,Marine Lines,-1,0
3,3,4,D,Grant Road,-1,0
4,4,5,E,Byculla,-1,0
5,5,6,F/N,Matunga,-1,0
6,6,7,F/S,Parel,-1,0
7,7,8,G/N,Dadar/Plaza,-1,0
8,8,9,G/S,Elphinstone,-1,0
9,9,10,H/E,Khar/Santacruz,-1,0


In [11]:
employment_nonhd = df[["wardIndex", "wardNo", "BMC_ID", "wardName", "nonslumDwePop", "#Totalnonslumworkers"]].sort_values(
    'wardNo'
).rename(columns={"nonslumDwePop": "totalPopulation","#Totalnonslumworkers":"Employed" })


employment_hd = df[["wardIndex", "wardNo", "BMC_ID", "wardName", "slumDwePop", "#TotalSlumworkers"]].sort_values(
    'wardNo'
).rename(columns={"slumDwePop": "totalPopulation","#TotalSlumworkers":"Employed" })

employment_hd["wardNo"] = employment_hd["wardNo"] + 24
employment_hd["wardIndex"] = employment_hd["wardIndex"] + 24
employment_hd["BMC_ID"] = employment_hd["BMC_ID"] + "_hd"
employment_hd["wardName"] = employment_hd["wardName"] + "_hd"

employment_nonhd["hd_flag"] = 0
employment_hd["hd_flag"] = 1


employment = pd.concat([employment_nonhd, employment_hd], axis=0)
employment["totalPopulation"] = employment["totalPopulation"].astype(int)
employment["Employed"] = employment["Employed"].astype(int)
employment["Unemployed"] = employment["totalPopulation"] - employment["Employed"]

checkRowOrdering(employment, 48, "employment")
employment

,wardIndex,wardNo,BMC_ID,wardName,totalPopulation,Employed,hd_flag,Unemployed
0,0,1,A,Colaba,162732,73717,0,89015
1,1,2,B,Sandhurst Road,114579,45258,0,69321
2,2,3,C,Marine Lines,149590,71354,0,78236
3,3,4,D,Grant Road,312167,137665,0,174502
4,4,5,E,Byculla,269092,109251,0,159841
5,5,6,F/N,Matunga,290906,114326,0,176580
6,6,7,F/S,Parel,180844,70710,0,110134
7,7,8,G/N,Dadar/Plaza,237365,98981,0,138384
8,8,9,G/S,Elphinstone,253443,106699,0,146744
9,9,10,H/E,Khar/Santacruz,168316,69682,0,98634


In [17]:
OD_nonhd = pd.read_excel(masterfile,sheet_name="Inter-Ward Travel Data").drop([0]).drop(
    columns=['Dest (area)']
).rename(columns={'Unnamed: 0':'wardNo'})
OD_nonhd.loc[:, "wardNo"] = OD_nonhd["wardNo"].apply(lambda x: wardLetterToNo[x])
OD_nonhd


,wardNo,A,B,C,D,E,F/N,F/S,G/N,G/S,...,P/S,R/C,R/N,R/S,L,M/E,M/W,N,S,T
1,1,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0.001887,0.003867,0.001392,0.001377,0.002342,0.004786,0.002872,0.005760,0.014181,0.010059
2,2,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0.001887,0.003867,0.001392,0.001377,0.002342,0.004786,0.002872,0.005760,0.014181,0.010059
3,3,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0.001887,0.003867,0.001392,0.001377,0.002342,0.004786,0.002872,0.005760,0.014181,0.010059
4,4,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0.001887,0.003867,0.001392,0.001377,0.002342,0.004786,0.002872,0.005760,0.014181,0.010059
5,5,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0.001887,0.003867,0.001392,0.001377,0.002342,0.004786,0.002872,0.005760,0.014181,0.010059
6,6,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0.003774,0.007734,0.002784,0.002754,0.004684,0.009572,0.005744,0.001920,0.004727,0.003353
7,7,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0.003774,0.007734,0.002784,0.002754,0.004684,0.009572,0.005744,0.001920,0.004727,0.003353
8,8,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0.003774,0.007734,0.002784,0.002754,0.004684,0.009572,0.005744,0.001920,0.004727,0.003353
9,9,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0.003774,0.007734,0.002784,0.002754,0.004684,0.009572,0.005744,0.001920,0.004727,0.003353
10,10,0.028406,0.005684,0.004088,0.015001,0.016821,0.014100,0.015185,0.010845,0.009870,...,0.009435,0.019335,0.006960,0.006885,0.004684,0.009572,0.005744,0.001920,0.004727,0.003353


In [20]:
OD_nonhd = pd.read_excel(masterfile,sheet_name="Inter-Ward Travel Data")
OD_nonhd = OD_nonhd.drop([0])
OD_nonhd = OD_nonhd.drop(columns=['Dest (area)'])
OD_nonhd= OD_nonhd.rename(columns={'Unnamed: 0':'wardNo'})
OD_nonhd["wardNo"] = OD_nonhd["wardNo"].apply(lambda x: wardLetterToNo[x])

cols = [a for a in OD_nonhd.columns if a != "wardNo"]
for col in cols:
    OD_nonhd[f"{col}_hd"] = 0
OD_hd = OD_nonhd.copy()
OD_hd["wardNo"] = OD_hd["wardNo"] + 24

OD = pd.concat([OD_nonhd, OD_hd], axis=0).sort_values('wardNo')

cols= [a for a in OD.columns if a != "wardNo"]
OD.rename(columns={a: wardLetterToNo[a] for a in cols}, inplace=True)

cols = [a for a in OD.columns if a != 'wardNo']
OD = OD[["wardNo"] + sorted(cols)].reset_index(drop=True)
OD

,wardNo,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,1,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
1,2,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
2,3,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
3,4,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
4,5,0.340872,0.068208,0.049056,0.180012,0.201852,0.016920,0.018222,0.013014,0.011844,...,0,0,0,0,0,0,0,0,0,0
5,6,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
6,7,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
7,8,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
8,9,0.089276,0.017864,0.012848,0.047146,0.052866,0.186120,0.200442,0.143154,0.130284,...,0,0,0,0,0,0,0,0,0,0
9,10,0.028406,0.005684,0.004088,0.015001,0.016821,0.014100,0.015185,0.010845,0.009870,...,0,0,0,0,0,0,0,0,0,0


In [21]:
demographics.to_csv('demographics.csv', index=False)
employment.to_csv('employment.csv',index=False)
households.to_csv('households.csv', index=False)
OD.to_csv('ODMatrix.csv', index=False)